In [1]:
# pipenv install yfinance==0.2.44 crewai==0.70.1 crewai-tools=0.12.1 langchain==0.2.16 langchain-community==0.2.17 langchain-openai==0.1.25 duckduckgo_search==6.3.0 ipykernel

In [14]:
import json
import os
from datetime import datetime, timedelta

from crewai import Agent, Task, Crew, Process

from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from crewai_tools import CSVSearchTool

import yfinance as yf

In [ ]:
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [4]:
csvWalletTool = CSVSearchTool(csv='./Wallet/wallet.csv')

c:\Users\felipe.fardo\.virtualenvs\ai-agents-crew-ai--UwK1Ags\Lib\site-packages\crewai_tools\tools\base_tool.py:93: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for arg, attribute in self.args_schema.schema()["properties"].items():


In [5]:
customerManager = Agent(
  role="Customer Stocks Manager",
  goal="Get the customer question about the stock {ticket} and search the customer wallet CSV file for the stocks.",
  backstory="""You're the manager of the customer investiment wallet.
  You are the client first contact and you provide the other analystis with the necessary stock tocket and wallet information
  """,
  verbose=True,
  llm=llm,
  max_iter=5,
  tools=[csvWalletTool],
  allow_delegation=False,
  memory=True
)

c:\Users\felipe.fardo\.virtualenvs\ai-agents-crew-ai--UwK1Ags\Lib\site-packages\langchain_core\tools\structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.args_schema.schema()["properties"]


In [6]:
getCustomerWallet = Task(
  description="""Use the customer question and find the {ticket} in the CSV File.
  Provide if the stock is in the customer wallet and if it is, provide with the mean price he paid
  and the total numbers of stocks owned.
  """,
  expected_output="If the customer own the stocks, provide the mean price paid and the total stock numbers",
  agent=customerManager
)

In [7]:
stocketPriceAnalyst = Agent(
  role="Senior Stock Analyst",
  goal="Find the {ticket} stock price and analyses price trends. Compre with the price that the customer paid.",
  backstory="""You're a highly experienced in analyzing the price of specific stocks and make predictions about its future price;""",
  verbose=True,
  llm=llm,
  max_iter=5,
  tools=[csvWalletTool],
  allow_delegation=False,
  memory=True
)

c:\Users\felipe.fardo\.virtualenvs\ai-agents-crew-ai--UwK1Ags\Lib\site-packages\langchain_core\tools\structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.args_schema.schema()["properties"]


In [17]:
# Criando Yahoo Finance Tool
def fetch_stock_price(ticket):
  end_date = datetime.today()
  start_date = end_date - timedelta(days=365)
  stock = yf.download(ticket, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
  return stock


yahoo_finance_tool = Tool(
  name='Yahoo Finance Tool',
  description='Fetches stocks prices for {ticket} from the last about a specific company from the Yahoo Finance API',
  func=lambda ticket: fetch_stock_price(ticket)
)

In [18]:
result = fetch_stock_price("AAPL")

2025-04-23 11:04:55,879 - 12160 - base.py-base:128 - ERROR: Failed to get ticker 'AAPL' reason: Expecting value: line 1 column 1 (char 0)


[*********************100%***********************]  1 of 1 completed
2025-04-23 11:04:55,894 - 24172 - multi.py-multi:171 - ERROR: 
1 Failed download:
2025-04-23 11:04:55,894 - 24172 - multi.py-multi:184 - ERROR: ['AAPL']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [12]:
result

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
getStockPrice = Task(
  description="Analyze the stock {ticket} price history and create a price trend analyses of up, down or sideways",
  expected_output="Specify the current trend stocks price - Up, down or sideway. eg. stock='AAPL, price UP'.",
  tools=[yahoo_finance_tool],
  agent=stocketPriceAnalyst
)